<a href="https://colab.research.google.com/github/playeredlc/treinamento-h2ia/blob/master/Busca-Adversarial/minimax_damas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Adversarial Search
# Minimax implementation to play the game of Checkers.

In [2]:
import numpy as np
import math
from copy import deepcopy

In [3]:
class Piece:
  def __init__(self, color, location=None):
    self.is_king = False
    self.color = color
    self.location = location
    self.available_moves = None

  def get_location(self):
    return self.location

  def set_location(self, new_location):
    self.location = new_location

  def belongs_to(self, player_color):
    return (self.color == player_color)
  
  def promote_to_king(self):
    self.is_king = True
  
  def set_available_moves(self, moves):
    self.available_moves = moves


In [4]:
class Board:
  def __init__(self, state=None, num_pieces=12, size=8):
    self.size = size
    self.num_pieces = num_pieces
    self.white_counter = num_pieces
    self.red_counter = num_pieces
    self.white_kings = 0
    self.red_kings = 0
    
    if(not state):
      self.state = self.initialize_board()
      self.white_counter = self.count_whites()
      self.red_counter = self.count_reds()
    else:
      self.state = state
      self.white_counter = self.count_whites()
      self.red_counter = self.count_reds()
      
  def initialize_board(self):
    # initialize empty board (dot indicating empty position)
    empty = Piece('.')
    board = [[empty]*self.size for _ in range(self.size)]

    # initialize player 1 (white)    
    for i in range(3):
      if(i%2==0):
        for j in range(self.size):
          if(j%2!=0):
            # even row, odd column
            board[i][j] = Piece('W', [i,j])
      else:
        for j in range(self.size):
          if(j%2==0):
            board[i][j] = Piece('W', [i,j])    
    #initialize player 2 (red)
    for i in range(self.size-1, self.size-4, -1):
      if(i%2==0):
        for j in range(self.size):
          if(j%2!=0):
            board[i][j] = Piece('R', [i,j])
      else:
        for j in range(self.size):
          if(j%2==0):
            board[i][j] = Piece('R', [i,j])
    
    return board

  def count_whites(self):
    whites = 0
    for row in self.state:
      whites += sum(piece.color == 'W' for piece in row)
    
    return whites

  def count_reds(self):
    reds = 0
    for row in self.state:
      reds += sum(piece.color == 'R' for piece in row)
    
    return reds

  def print_board(self, state=None, last_move=None, last_piece=None):
    if(not state):
      state = self.state

    for row_index, row in enumerate(state):
      print('\n\n')
      for piece_index, piece in enumerate(row):
        if(piece == last_piece or [row_index, piece_index] == last_move):
          print('\t|', piece.color, "|", end="")
        else:
          print('\t', piece.color, end="")
    print('\n')
  
  def evaluate(self):
    utility = (self.white_counter - self.red_counter) + (self.white_kings * 0.5 - self.red_kings * 0.5)
    return utility

  def get_pieces(self, player_id):
    pieces_list = []

    for row in self.state:
      for piece in row:
        if (type(piece) == Piece and piece.color == player_id):
          pieces_list.append(piece)
    
    return pieces_list

  def set_valid_moves(self, player_id):
    moves = []
    pieces = self.get_pieces(player_id)
    
    if(player_id == 'W'):
      row_direction = 1
    else:
      row_direction = -1
    
    for piece in pieces:
      if(piece.is_king):
        new_rows = [piece.location[0] + row_direction, piece.location[0] - row_direction]
      else:
        new_rows = [piece.location[0] + row_direction]
      new_cols = [piece.location[1] + 1, piece.location[1] - 1]

      for new_row in new_rows:
        for new_col in new_cols:
          if(self._out_of_bound(new_row)):
            # invalid move
            continue
          elif(self._out_of_bound(new_col)):
            # invalid move
            continue
          elif(self.state[new_row][new_col].color == player_id):
            # invalid move
            continue
          
          elif((self.state[new_row][new_col].color == 'W' and player_id == 'R') or \
               (self.state[new_row][new_col].color == 'R' and player_id == 'W')):
            # check if killing is possible                  
            if(new_row > piece.location[0]):
              aux_row = new_row + 1
            else:
              aux_row = new_row - 1              
            if(new_col > piece.location[1]):
              aux_col = new_col + 1
            else:
              aux_col = new_col - 1

            if(self._out_of_bound(aux_row) or self._out_of_bound(aux_col)):
              # invalid move
              continue
            elif(self.state[aux_row][aux_col].color != '.'):                                     
              # invalid move
              continue
            else:
              # kill it and jump
              dead_piece = [new_row, new_col]
              killed = True

              new_valid_pos = [aux_row, aux_col, killed, dead_piece]
              moves.append(new_valid_pos)
              continue
          else:
            # jump without killing any piece
            killed = False
            new_valid_pos = [new_row, new_col, killed, []]
            moves.append(new_valid_pos)
            continue
      
      piece.set_available_moves(moves)
      moves = []
          
  def _out_of_bound(self, new_pos):
    if(new_pos >= self.size or new_pos < 0):
      return True
    
    return False

  def simulate_move(self, piece, move):    
    r, c = piece.location[0], piece.location[1]
    new_r, new_c = move[0], move[1]
    killed = move[2]

    new_state = deepcopy(self.state)
    new_piece = deepcopy(piece)
    if(killed):
      dead_piece = move[3]
      new_state[dead_piece[0]][dead_piece[1]] = Piece('.')

    new_state[r][c] = Piece('.')
    new_state[new_r][new_c] = new_piece
    new_piece.set_location([new_r, new_c])

    return new_state

  def move_piece(self, piece, move):
    r, c = piece.location[0], piece.location[1]
    new_r, new_c = move[0], move[1]
    killed = move[2]
    
    if(killed):
      dead_piece = move[3]
      if(self.state[dead_piece[0]][dead_piece[1]].color == 'W'):
        self.white_counter -= 1
      else:
        self.red_counter -= 1
      self.state[dead_piece[0]][dead_piece[1]] = Piece('.')
    
    if(piece.color == 'W' and not piece.is_king and new_r == self.size-1):
      piece.is_king = True
    elif(piece.color == 'R' and not piece.is_king and new_r == 0):
      piece.is_king = True
    
    self.state[r][c] = Piece('.')
    self.state[new_r][new_c] = piece
    piece.set_location([new_r, new_c])

  def end_game(self):
    if(self.white_counter == 0):
      return 'R'
    elif(self.red_counter == 0):
      return 'W'
    
    return False
  
  def print_score(self):
    print(f'WHI {self.white_counter} x {self.red_counter} RED\n')
  
  def print_winner(self, winner_color):
    if(winner_color == 'W'):
      print(f'END OF THE MATCH.\nPLAYER 1 (WHITE) WINS!\n')
    elif(winner_color == 'R'):
      print(f'END OF THE MATCH.\nPLAYER 2 (RED) WINS!\n')
    else:
      return None


In [5]:
class Match:
  def __init__(self, board, depth=3):
    self.current_board = board
    self.depth = depth

  def start(self):
    board = self.current_board
    while(True):
      # WHITE PLAYS
      utility, white_move, white_piece = minimax(board, None, None, depth, True)
      
      # prev location used for better visualization
      previous_location = white_piece.location
      
      # move piece based on the move minimax returned
      board.move_piece(white_piece, white_move)
      
      # print movement on screen
      print("Player 1 (white) plays:")
      board.print_board(last_piece=white_piece, last_move=previous_location)
      board.print_score()
      print("IT IS RED'S TURN!")

      # check for a winner
      winner = board.end_game()
      if(winner):
        board.print_winner(winner)
        break

      # RED PLAYS
      utility, red_move, red_piece = minimax(board, None, None, depth, False)
      previous_location = red_piece.location
      board.move_piece(red_piece, red_move)
      print("Player 2 (red) plays:")
      board.print_board(last_piece=red_piece, last_move=previous_location)
      board.print_score()
      print("IT IS WHITE'S TURN!")

      # check for a winner
      winner = board.end_game()
      if(winner):
        board.print_winner(winner)
        break


In [6]:
def minimax(board, move, piece, depth, is_max):
  if(depth == 0 or board.end_game()):    
    return board.evaluate(), move, piece
  if(is_max):
    # max playing
    max_utility = -math.inf
    max_move = None
    max_piece = None

    pieces = board.get_pieces('W')
    board.set_valid_moves('W')
    for piece in pieces:
      for move in piece.available_moves:
        simulated_board = Board(board.simulate_move(piece, move))
        
        utility, _, _ = minimax(simulated_board, move, piece, depth-1, False)
        
        max_utility = max(max_utility, utility)        
        if(max_utility == utility and utility != None):
          max_move = move
          max_piece = piece
    
    return max_utility, max_move, max_piece

  else:
    # min playing
    min_utility = math.inf
    min_move = None
    min_piece = None

    pieces = board.get_pieces('R')
    board.set_valid_moves('R')
    for piece in pieces:
      for move in piece.available_moves:
        simulated_board = Board(board.simulate_move(piece, move))
        
        utility, _, _ = minimax(simulated_board, move, piece, depth-1, True)

        min_utility = min(min_utility, utility)
        if(min_utility == utility and utility != None):
          min_move = move
          min_piece = piece
      
    return min_utility, min_move, min_piece


In [7]:
board = Board()
depth = 3
match = Match(board, depth)


In [8]:
match.start()

Player 1 (white) plays:



	 .	 W	 .	 W	 .	 W	 .	 W


	 W	 .	 W	 .	 W	 .	 W	 .


	 .	 W	 .	 W	 .	 W	 .	| . |


	 .	 .	 .	 .	 .	 .	| W |	 .


	 .	 .	 .	 .	 .	 .	 .	 .


	 R	 .	 R	 .	 R	 .	 R	 .


	 .	 R	 .	 R	 .	 R	 .	 R


	 R	 .	 R	 .	 R	 .	 R	 .

WHI 12 x 12 RED

IT IS RED'S TURN!
Player 2 (red) plays:



	 .	 W	 .	 W	 .	 W	 .	 W


	 W	 .	 W	 .	 W	 .	 W	 .


	 .	 W	 .	 W	 .	 W	 .	 .


	 .	 .	 .	 .	 .	 .	 W	 .


	 .	 .	 .	 .	 .	| R |	 .	 .


	 R	 .	 R	 .	 R	 .	| . |	 .


	 .	 R	 .	 R	 .	 R	 .	 R


	 R	 .	 R	 .	 R	 .	 R	 .

WHI 12 x 12 RED

IT IS WHITE'S TURN!
Player 1 (white) plays:



	 .	 W	 .	 W	 .	 W	 .	 W


	 W	 .	 W	 .	 W	 .	 W	 .


	 .	 W	 .	 W	 .	 W	 .	 .


	 .	 .	 .	 .	 .	 .	| . |	 .


	 .	 .	 .	 .	 .	 R	 .	| W |


	 R	 .	 R	 .	 R	 .	 .	 .


	 .	 R	 .	 R	 .	 R	 .	 R


	 R	 .	 R	 .	 R	 .	 R	 .

WHI 12 x 12 RED

IT IS RED'S TURN!
Player 2 (red) plays:



	 .	 W	 .	 W	 .	 W	 .	 W


	 W	 .	 W	 .	 W	 .	 W	 .


	 .	 W	 .	 W	 .	 W	 .	 .


	 .	 .	 .	 .	 .	 .	 .	 .


	 .	 .	 .	 .	 .	 R

In [9]:
# 
# TODO
# improve evaluation function
# implement alpha-beta prunning
#